In [27]:
import os
import json
import random
from tqdm import tqdm
import time
import psutil
import gc
import numpy as np
from tabulate import tabulate
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch_geometric.nn import GATConv
from torch_geometric.data import Data as GraphData
from torch_geometric.utils import dense_to_sparse
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo, nvmlShutdown
from codecarbon import EmissionsTracker

from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch_geometric.nn import GATConv
from torch_geometric.utils import dense_to_sparse

In [28]:
def set_seed(seed: int = 42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % (2**32)
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [ ]:
final_output_json='../../data/processed/final_annotations_without_occluded.json'
image_directory = '../../data/images'

test_ratio = 0.2
valid_ratio = 0.1
random_seed = 42

with open(final_output_json, 'r') as f:
    annotations = json.load(f)

image_filenames = list(annotations['images'].keys())[:100]

random.seed(random_seed)
random.shuffle(image_filenames)

num_test = int(len(image_filenames) * test_ratio)
test_images = image_filenames[:num_test]
train_images = image_filenames[num_test:]
num_valid = int(len(train_images) * valid_ratio)
valid_images = train_images[:num_valid]

train_annotations = {
    'all_parts': annotations['all_parts'],
    'images': {img_name: annotations['images'][img_name] for img_name in train_images}
}

valid_annotations = {
    'all_parts': annotations['all_parts'],
    'images': {img_name: annotations['images'][img_name] for img_name in valid_images}
}

test_annotations = {
    'all_parts': annotations['all_parts'],
    'images': {img_name: annotations['images'][img_name] for img_name in test_images}
}


In [30]:
class BikePartsDetectionDataset(Dataset):
    def __init__(self, annotations_dict, image_dir, transform=None, augment=True, target_size=(640, 640)):
        self.all_parts = annotations_dict['all_parts']
        self.part_to_idx = {part: idx + 1 for idx, part in enumerate(self.all_parts)}
        self.idx_to_part = {idx + 1: part for idx, part in enumerate(self.all_parts)}
        self.image_data = annotations_dict['images']
        self.image_filenames = list(self.image_data.keys())
        self.image_dir = image_dir
        self.transform = transform
        self.augment = augment
        self.target_size = target_size

    def __len__(self):
        return len(self.image_filenames) * (2 if self.augment else 1)

    def apply_augmentation(self, image, boxes):
        if random.random() < 0.5:
            image = transforms.functional.hflip(image)
            w = image.width
            boxes = boxes.clone()
            boxes[:, [0, 2]] = w - boxes[:, [2, 0]]

        if random.random() < 0.8:
            image = transforms.functional.adjust_brightness(image, brightness_factor=random.uniform(0.6, 1.4))
        if random.random() < 0.8:
            image = transforms.functional.adjust_contrast(image, contrast_factor=random.uniform(0.6, 1.4))
        if random.random() < 0.5:
            image = transforms.functional.adjust_saturation(image, saturation_factor=random.uniform(0.7, 1.3))

        return image, boxes

    def __getitem__(self, idx):
        real_idx = idx % len(self.image_filenames)
        do_augment = self.augment and (idx >= len(self.image_filenames))

        img_filename = self.image_filenames[real_idx]
        img_path = os.path.join(self.image_dir, img_filename)

        image = Image.open(img_path).convert('RGB')
        orig_width, orig_height = image.size

        annotation = self.image_data[img_filename]
        available_parts_info = annotation['available_parts']
        missing_parts_names = annotation.get('missing_parts', [])

        boxes = []
        labels = []

        for part_info in available_parts_info:
            part_name = part_info['part_name']
            bbox = part_info['absolute_bounding_box']
            xmin = bbox['left']
            ymin = bbox['top']
            xmax = xmin + bbox['width']
            ymax = ymin + bbox['height']
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(self.part_to_idx[part_name])

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        if do_augment:
            image, boxes = self.apply_augmentation(image, boxes)

        image = transforms.functional.resize(image, self.target_size)
        new_width, new_height = self.target_size
        scale_x = new_width / orig_width
        scale_y = new_height / orig_height
        boxes[:, [0, 2]] *= scale_x
        boxes[:, [1, 3]] *= scale_y

        image = transforms.functional.to_tensor(image)

        missing_labels = torch.tensor(
            [self.part_to_idx[part] for part in missing_parts_names],
            dtype=torch.int64
        )

        target = {
            'boxes': boxes,
            'labels': labels,
            'missing_labels': missing_labels,
            'image_id': torch.tensor([real_idx])
        }

        return image, target

In [31]:
train_dataset = BikePartsDetectionDataset(
    annotations_dict=train_annotations,
    image_dir=image_directory,
    augment=False
)

valid_dataset = BikePartsDetectionDataset(
    annotations_dict=valid_annotations,
    image_dir=image_directory,
    augment=False
)

test_dataset = BikePartsDetectionDataset(
    annotations_dict=test_annotations,
    image_dir=image_directory,
    augment=False
)

train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4,
    collate_fn=lambda batch: tuple(zip(*batch))
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    collate_fn=lambda batch: tuple(zip(*batch))
)

test_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    collate_fn=lambda batch: tuple(zip(*batch))
)

In [32]:
def evaluate_model(model, data_loader, part_to_idx, device):
    model.eval()

    all_parts_set = set(part_to_idx.values())
    results_per_image = []

    for images, targets in tqdm(data_loader, desc="Evaluating"):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        with torch.no_grad():
            predictions = model(images)

        for i in range(len(images)):
            pred_parts = set(predictions[i]['labels'].cpu().numpy().tolist())
            true_missing_parts = set(targets[i]['missing_labels'].cpu().numpy().tolist())
            image_id = targets[i]['image_id'].item()

            predicted_missing_parts = all_parts_set - pred_parts

            results_per_image.append({
                'image_id': image_id,
                'predicted_missing_parts': predicted_missing_parts,
                'true_missing_parts': true_missing_parts
            })

    return results_per_image


def part_level_evaluation(results, part_to_idx, idx_to_part):
    parts = list(part_to_idx.values())

    Y_true = np.array([[1 if p in r['true_missing_parts'] else 0 for p in parts] for r in results])
    Y_pred = np.array([[1 if p in r['predicted_missing_parts'] else 0 for p in parts] for r in results])

    micro_f1 = f1_score(Y_true, Y_pred, average='micro', zero_division=0)
    macro_f1 = f1_score(Y_true, Y_pred, average='macro', zero_division=0)

    FN = np.logical_and(Y_true==1, Y_pred==0).sum()
    TP = np.logical_and(Y_true==1, Y_pred==1).sum()
    FP = np.logical_and(Y_true==0, Y_pred==1).sum()

    N_images = len(results)
    miss_rate = FN/(FN+TP) if (FN+TP)>0 else 0
    fppi = FP/N_images

    overall_acc = accuracy_score(Y_true.flatten(), Y_pred.flatten())
    overall_prec = precision_score(Y_true.flatten(), Y_pred.flatten(), zero_division=0)
    overall_rec = recall_score(Y_true.flatten(), Y_pred.flatten(), zero_division=0)
    overall_f1 = f1_score(Y_true.flatten(), Y_pred.flatten(), zero_division=0)
    print(f"Micro-F1: {micro_f1:.4f}, Macro-F1: {macro_f1:.4f}")
    print(f"Miss Rate: {miss_rate:.4f}, FPPI: {fppi:.4f}")
    print(f"Overall Acc: {overall_acc:.4f}, Precision: {overall_prec:.4f}, Recall: {overall_rec:.4f}, F1: {overall_f1:.4f}")
    
    table=[]
    for j,p in enumerate(parts):
        acc = accuracy_score(Y_true[:,j], Y_pred[:,j])
        prec = precision_score(Y_true[:,j], Y_pred[:,j], zero_division=0)
        rec = recall_score(Y_true[:,j], Y_pred[:,j], zero_division=0)
        f1s = f1_score(Y_true[:,j], Y_pred[:,j], zero_division=0)
        table.append([idx_to_part[p], f"{acc:.3f}", f"{prec:.3f}", f"{rec:.3f}", f"{f1s:.3f}"])
    print(tabulate(table, headers=["Part","Acc","Prec","Rec","F1"], tablefmt="fancy_grid"))


In [ ]:
class GNNHead(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_heads=4):
        super().__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=num_heads, concat=True)
        self.relu = nn.ReLU()
        self.gat2 = GATConv(hidden_channels * num_heads, out_channels, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = self.gat1(x, edge_index)
        x = self.relu(x)
        x = self.gat2(x, edge_index)
        return x

class GraphRCNNWithDualPrediction(nn.Module):
    def __init__(self, num_classes, hidden_channels=512):
        super().__init__()

        self.detector = fasterrcnn_mobilenet_v3_large_fpn(weights="DEFAULT")
        in_features = self.detector.roi_heads.box_predictor.cls_score.in_features
        self.detector.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

        self.gnn_head = GNNHead(in_channels=in_features, hidden_channels=hidden_channels, out_channels=num_classes)
        self.num_classes = num_classes
        self.gnn_loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, images, targets=None):
        if self.training:
            detector_losses = self.detector(images, targets)
            
            with torch.no_grad():
                image_list, _ = self.detector.transform(images, targets)
                features = self.detector.backbone(image_list.tensors)
            image_shapes = image_list.image_sizes
            proposals = [t["boxes"].detach() for t in targets]

            roi_pooled = self.detector.roi_heads.box_roi_pool(features, proposals, image_shapes)
            roi_features = self.detector.roi_heads.box_head(roi_pooled)

            gnn_losses = 0.0
            start = 0
            gnn_loss_fn = nn.CrossEntropyLoss()  # define once outside loop

            for i, boxes in enumerate(proposals):
                num_nodes = boxes.shape[0]
                if num_nodes == 0:
                    continue

                roi_feat = roi_features[start:start + num_nodes]
                edge_index = self._build_fully_connected_edges(num_nodes).to(roi_feat.device)

                gnn_logits = self.gnn_head(roi_feat, edge_index)  # [num_boxes, num_classes]

                part_labels = targets[i]["labels"].to(roi_feat.device)  # [num_boxes]
                gnn_loss = gnn_loss_fn(gnn_logits, part_labels)  # CrossEntropyLoss

                gnn_losses += gnn_loss
                start += num_nodes

            total_loss = sum(detector_losses.values()) + gnn_losses

            return {
                **detector_losses,
                "loss_gnn": gnn_losses,
                "loss_total": total_loss
            }

        else:
            detections = self.detector(images)
            all_outputs = []

            for img_idx, det in enumerate(detections):
                boxes = det["boxes"]

                if boxes.shape[0] == 0:
                    all_outputs.append({
                        "boxes": boxes,
                        "scores": det["scores"],
                        "labels": torch.tensor([], dtype=torch.long, device=boxes.device)
                    })
                    continue

                features = self._get_roi_features(images[img_idx].unsqueeze(0), boxes)

                num_nodes = features.shape[0]
                edge_index = self._build_fully_connected_edges(num_nodes).to(features.device)
                gnn_logits = self.gnn_head(features, edge_index)  # [num_boxes, num_classes]

                # For each box, predict a single label via argmax over classes
                per_box_labels = gnn_logits.argmax(dim=1)  # [num_boxes] -> int labels
                unique_labels = per_box_labels.unique()

                final_output = {
                    "boxes": boxes,
                    "scores": det["scores"],
                    "labels": unique_labels  # predicted part classes present in image
                }
                all_outputs.append(final_output)

            return all_outputs


    def _get_roi_features(self, image, boxes):
        with torch.no_grad():
            features = self.detector.backbone(image.tensors if hasattr(image, 'tensors') else image)

        image_shapes = [img.shape[-2:] for img in image]
        roi_pooled = self.detector.roi_heads.box_roi_pool(features, [boxes], image_shapes)
        roi_features = self.detector.roi_heads.box_head(roi_pooled)
        return roi_features

    def _build_fully_connected_edges(self, num_nodes):
        adj = torch.ones((num_nodes, num_nodes)) - torch.eye(num_nodes)
        return dense_to_sparse(adj)[0]


In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = len(train_dataset.all_parts) + 1
model = GraphRCNNWithDualPrediction(num_classes=num_classes, hidden_channels=512)
model.to(device)

learning_rate = 1e-4
weight_decay = 1e-4

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=learning_rate, weight_decay=weight_decay)

if torch.cuda.is_available():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)

num_epochs = 1
best_macro_f1 = 0
epochs_without_improvement = 0
patience = 3
for epoch in range(num_epochs):

    with EmissionsTracker(log_level="critical", save_to_file=False) as tracker:

        model.train()

        batch_times = []
        gpu_memories = []
        cpu_memories = []

        with tqdm(train_loader, unit="batch", desc=f"Epoch {epoch+1}/{num_epochs}") as tepoch:
            for images, targets in tepoch:
                start_time = time.time()

                images = [image.to(device) for image in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                optimizer.zero_grad()
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())
                losses.backward()



                optimizer.step()

                end_time = time.time()
                inference_time = end_time - start_time
                batch_times.append(inference_time)

                if torch.cuda.is_available():
                    mem_info = nvmlDeviceGetMemoryInfo(handle)
                    gpu_mem_used = mem_info.used / (1024 ** 2)
                    gpu_memories.append(gpu_mem_used)
                else:
                    gpu_mem_used = 0

                cpu_mem_used = psutil.virtual_memory().used / (1024 ** 2)
                cpu_memories.append(cpu_mem_used)

                tepoch.set_postfix({
                    "loss": f"{losses.item():.4f}",
                    "time (s)": f"{inference_time:.3f}",
                    "GPU Mem (MB)": f"{gpu_mem_used:.0f}",
                    "CPU Mem (MB)": f"{cpu_mem_used:.0f}"
                })

                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()


    energy_consumption = tracker.final_emissions_data.energy_consumed
    co2_emissions = tracker.final_emissions

    avg_time = sum(batch_times) / len(batch_times)
    max_gpu_mem = max(gpu_memories) if gpu_memories else 0
    max_cpu_mem = max(cpu_memories)

    table = [
        ["Epoch", epoch + 1],
        ["Final Loss", f"{losses.item():.4f}"],
        ["Average Batch Time (sec)", f"{avg_time:.4f}"],
        ["Average GPU Memory Usage (MB)", f"{max_gpu_mem:.2f}"],
        ["Average CPU Memory Usage (MB)", f"{max_cpu_mem:.2f}"],
        ["Energy Consumption (kWh)", f"{energy_consumption:.4f} kWh"],
        ["CO₂ Emissions (kg)", f"{co2_emissions:.4f} kg"],
    ]

    print(tabulate(table, headers=["Metric", "Value"], tablefmt="pretty"))

    print(f"\nEvaluating on validation set after Epoch {epoch + 1}...")
    results_per_image = evaluate_model(model, valid_loader, train_dataset.part_to_idx, device)

    parts = list(train_dataset.part_to_idx.values())
    Y_true = np.array([[1 if p in r['true_missing_parts'] else 0 for p in parts] for r in results_per_image])
    Y_pred = np.array([[1 if p in r['predicted_missing_parts'] else 0 for p in parts] for r in results_per_image])
    macro_f1 = f1_score(Y_true, Y_pred, average='macro', zero_division=0)

    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        epochs_without_improvement = 0
        torch.save(model.state_dict(), f"../../models/graph_rcnn/graphrcnn_MobileNet_baseline_model.pth")
        print(f"Saved new best model (macro-F1: {macro_f1:.4f})")
    else:
        epochs_without_improvement += 1
        print(f"No improvement in macro-F1 for {epochs_without_improvement} epoch(s)")

        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered (no improvement for {patience} epochs)")
            break


if torch.cuda.is_available():
    nvmlShutdown()

Epoch 1/1: 100%|██████████| 100/100 [12:15<00:00,  7.35s/batch, loss=25.9231, time (s)=6.872, GPU Mem (MB)=0, CPU Mem (MB)=9767]


+-------------------------------+------------+
|            Metric             |   Value    |
+-------------------------------+------------+
|             Epoch             |     1      |
|          Final Loss           |  25.9231   |
|   Average Batch Time (sec)    |   7.0983   |
| Average GPU Memory Usage (MB) |    0.00    |
| Average CPU Memory Usage (MB) |  9958.03   |
|   Energy Consumption (kWh)    | 0.0034 kWh |
|      CO₂ Emissions (kg)       | 0.0009 kg  |
+-------------------------------+------------+

Evaluating on validation set after Epoch 1...


Evaluating:   0%|          | 0/10 [00:03<?, ?it/s]


ValueError: images is expected to be a list of 3d tensors of shape [C, H, W], got torch.Size([1, 3, 640, 640])

In [ ]:
model.load_state_dict(torch.load("../../models/graph_rcnn/graphrcnn_MobileNet_baseline_model.pth", map_location=device))
model.to(device)

results_per_image = evaluate_model(model, valid_loader, train_dataset.part_to_idx, device)

print(results_per_image)

part_level_evaluation(
    results_per_image, train_dataset.part_to_idx, train_dataset.idx_to_part
)


/tmp/ipykernel_57557/349410656.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../../models/graph_rcnn/graphrcnn_MobileNet_baseline_mod

[{'image_id': 0, 'predicted_missing_parts': {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22}, 'true_missing_parts': {1, 9, 16, 17, 18, 21, 22}}, {'image_id': 1, 'predicted_missing_parts': {1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22}, 'true_missing_parts': {1, 9, 13, 14, 19}}, {'image_id': 2, 'predicted_missing_parts': {1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22}, 'true_missing_parts': {2, 6, 7, 9, 11, 13, 16, 17, 19, 21, 22}}, {'image_id': 3, 'predicted_missing_parts': {1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22}, 'true_missing_parts': {2, 5, 7, 8, 9, 11, 16, 17, 18, 19, 21, 22}}, {'image_id': 4, 'predicted_missing_parts': {1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22}, 'true_missing_parts': {2, 5, 9, 11, 17, 18, 19, 21}}, {'image_id': 5, 'predicted_missing_parts': {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,